In [401]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [4]:
import pyodbc
conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=LAPTOP-OOBA2U3O\SQLEXPRESS;"
                      "Database=CollegeDB;"
                      "Trusted_Connection=yes;")

In [194]:
students = pd.read_sql('SELECT * FROM Students', conn)
courses = pd.read_sql('SELECT * FROM Courses', conn)
classrooms = pd.read_sql('SELECT * FROM Classrooms', conn)
departments = pd.read_sql('SELECT * FROM Departments', conn)
teachers = pd.read_sql('SELECT * FROM Teachers', conn)

In [515]:
courses_departments = pd.merge(courses, departments, how='inner', left_on='DepartmentID', right_on='DepartmentId')
courses_departments_classrooms = pd.merge(courses_departments, classrooms, how='inner', on='CourseId')

courses_classrooms = pd.merge(courses, classrooms, how='inner', on='CourseId')
students_courses_classrooms = pd.merge(students, courses_classrooms, how='inner', on='StudentId')

teachers_courses_classrooms = pd.merge(teachers, courses_classrooms, how='inner', on='TeacherId')

teachers_courses_departments_classrooms = pd.merge(teachers, courses_departments_classrooms, how='inner', on='TeacherId')

classrooms_students_L = pd.merge(classrooms, students, how='left', on='StudentId')
classrooms_students_courses_L = pd.merge(classrooms_students_L, courses, how='left', on='CourseId')

classrooms_students_courses_departments_L = pd.merge(classrooms_students_courses_L, departments, how='left', left_on='DepartmentID', right_on='DepartmentId')

In [525]:
# ---- 2a. Number of students by Department

courses_departments_classrooms

df_departments_students_no = courses_departments_classrooms.groupby('DepartmentName').agg({'StudentId':'nunique'})

df_departments_students_no.rename(columns={'StudentId':'students'}, inplace=True)

print (df_departments_students_no)

                students
DepartmentName          
Arts                 137
English               84
Science              200
Sport                 81


In [526]:
# ---- 2b. How many studenst has the English teacher by course and in total?
courses_departments_classrooms

df_english_courses_departments_classrooms = courses_departments_classrooms[courses_departments_classrooms['DepartmentId'] == 1]
df_english_courses_students_no = df_english_courses_departments_classrooms.groupby('CourseName').agg({'StudentId':'nunique'})

v_total_english_students = df_english_courses_departments_classrooms.agg({'StudentId':'nunique'}).astype(int)

df_english_courses_students_no.loc['Total']= v_total_english_students

df_english_courses_students_no.rename(columns={'StudentId':'students'}, inplace=True)

print(df_english_courses_students_no)

                      students
CourseName                    
Advanced English            24
English Begginers           32
Proffesional English        38
Total                       84


In [527]:
# ---- 2c. How many small (<22) and bigger (>=22) Classrooms has the Science Department?

courses_departments_classrooms

df_science_courses_departments_classrooms = courses_departments_classrooms[courses_departments_classrooms['DepartmentID'] == 2]

df_science_courses_students_no = df_science_courses_departments_classrooms.groupby('CourseName').agg({'StudentId':'nunique'})

df_science_courses_students_no['classroom_size'] = np.NaN

df_science_courses_students_no.loc[(df_science_courses_students_no['StudentId'] < 22), 'classroom_size'] = 'Small classrooms'
df_science_courses_students_no.loc[(df_science_courses_students_no['StudentId'] >= 22), 'classroom_size'] = 'Big classrooms'

df_classrooms_no_per_size = df_science_courses_students_no.groupby('classroom_size').agg({'StudentId': 'count'})

df_classrooms_no_per_size.rename(columns={'StudentId':'num_classrooms'}, inplace=True)

print(df_classrooms_no_per_size)

                  num_classrooms
classroom_size                  
Big classrooms                10
Small classrooms               6


In [541]:
# ---- 2d. How many students are by Gender?

df_students_by_gender = students.groupby('Gender').agg({'StudentId':'nunique'})
df_students_by_gender.rename(columns={'StudentId':'num_students'}, inplace=True)

print(df_students_by_gender)

        num_students
Gender              
F                165
M                115


In [529]:
# ---- 2e. In which courses the percentage of males / females are higher than 70% ?

students_courses_classrooms['Male'] = 0
students_courses_classrooms['Female'] = 0

students_courses_classrooms.loc[(students_courses_classrooms['Gender'] == 'M'), 'Male'] = 1
students_courses_classrooms.loc[(students_courses_classrooms['Gender'] == 'F'), 'Female'] = 1

courses_gender_no = students_courses_classrooms.groupby('CourseName').agg({'Male':sum, 'Female':sum})

courses_gender_no['ratio'] = courses_gender_no['Female'] / (courses_gender_no['Male'] + courses_gender_no['Female']) * 100

print(courses_gender_no[courses_gender_no['ratio'] > 70])

            Male  Female      ratio
CourseName                         
Sculpture      7      17  70.833333
Tenis          5      13  72.222222


In [531]:
# ---- 2f. getting the number of students with at least one degree with more than 80 per department

courses_departments_classrooms_f = courses_departments_classrooms.copy()

courses_departments_classrooms_f['students_80'] = np.NaN

courses_departments_classrooms_f.loc[(courses_departments_classrooms_f['degree'] > 80), 'students_80'] = courses_departments_classrooms_f['StudentId']

departments_grades = courses_departments_classrooms_f.groupby('DepartmentName').agg({'students_80':'nunique', 'StudentId':'nunique'})

departments_grades.rename(columns={'StudentId':'total_students'}, inplace=True)

departments_grades['students_80_pct'] = departments_grades['students_80'] / departments_grades['total_students'] * 100

print(departments_grades.round(3))

                students_80  total_students  students_80_pct
DepartmentName                                              
Arts                     61             137           44.526
English                  40              84           47.619
Science                 122             200           61.000
Sport                    39              81           48.148


In [533]:
# ---- 2g. How many students (n and %) have a degree lower than 60 by Department?

courses_departments_classrooms_g = courses_departments_classrooms.copy()

courses_departments_classrooms_g['students_60'] = np.NaN

courses_departments_classrooms_g.loc[(courses_departments_classrooms_g['degree'] < 60), 'students_60'] = courses_departments_classrooms_g['StudentId']

departments_grades = courses_departments_classrooms_g.groupby('DepartmentName').agg({'students_60':'nunique', 'StudentId':'nunique'})

departments_grades.rename(columns={'StudentId':'total_students'}, inplace=True)

departments_grades['students_60_pct'] = departments_grades['students_60'] / departments_grades['total_students'] * 100

print(departments_grades.round(3))

                students_60  total_students  students_60_pct
DepartmentName                                              
Arts                      3             137            2.190
English                   2              84            2.381
Science                   7             200            3.500
Sport                     1              81            1.235


In [534]:
# ---- 2h Rate in descending order the teachers by their student's mean degree.
teachers_courses_classrooms_h = teachers_courses_classrooms.copy()

teachers_courses_classrooms_h['Teacher'] = teachers_courses_classrooms_h[['FirstName', 'LastName']].agg(' '.join, axis=1)

#teachers_courses_classrooms_h.drop(['FirstName', 'LastName'], axis=1)

teachers_degree = teachers_courses_classrooms_h.groupby('Teacher').agg({'degree':'mean'})

print(teachers_degree.sort_values("degree", ascending=False).round(3))

                    degree
Teacher                   
 Darick Hess        81.433
 Jordyn Baron       80.729
 Heather Metcalf    80.440
 Zachary Healy      79.366
 Kaylea Monzingo    79.366
 Daniel Bland       78.908
 Keegan Price       78.815
 Conor Mckittrick   78.699
 Maxwell Romkee     78.674
 Terran Beaty       78.495
 Tanner Fitzsimons  78.443
 Mackenzie Olsen    78.383
 Ernest Stoyanov    78.316
 Madison Batan      77.744
 Katerina Young     77.407
 Mikayla Russell    77.134
 Amanda Dworkin     76.900
 Kaitlin Delude     76.765
 Jeffrey Batzel     76.738
 Robert Aivazian    76.516
 Caroline Knuesel   75.955
 Alexander Watts    75.668


In [535]:
# ---- 3a. Create a view that shows the courses, departments, teachers and number of students on each

teachers_courses_departments_classrooms_3a = teachers_courses_departments_classrooms.copy()
result = teachers_courses_departments_classrooms_3a.groupby(['CourseId', 'CourseName', 'DepartmentName', 'FirstName', 'LastName']).agg({'StudentId':'count'})

print(result)

                                                                    StudentId
CourseId CourseName           DepartmentName FirstName  LastName             
1.0      English Begginers    English         Conor     Mckittrick         32
2.0      Advanced English     English         Conor     Mckittrick         24
3.0      Proffesional English English         Conor     Mckittrick         38
4.0      Mathematics 101      Science         Amanda    Dworkin            18
5.0      Matematics Advanced  Science         Alexander Watts              10
6.0      Algebra              Science         Alexander Watts              12
7.0      Geometry             Science         Alexander Watts              14
8.0      Chemistry            Science         Jeffrey   Batzel             24
9.0      Physics              Science         Alexander Watts              12
10.0     Biology              Science         Jordyn    Baron              36
11.0     Genetics             Science         Terran    Beaty   

In [540]:
# ---- 3b. Create a view that shows each student, the number of courses taken, 
# ---- their mean degree by department and the total degree mean.

classrooms_students_courses_departments_L_3b = classrooms_students_courses_departments_L.copy()

general_degree_per_student = classrooms_students_courses_departments_L_3b.groupby('StudentId').agg({'degree':'mean'})

#general_degree_per_student.sort_values("StudentId", ascending=True)

courses_number_per_student = classrooms_students_courses_departments_L_3b.groupby('StudentId').agg({'CourseId':'count'})

degree_by_relevant_departments_per_student = classrooms_students_courses_departments_L_3b.groupby(['StudentId', 'FirstName',  'LastName', 'DepartmentName']).agg({'degree':'mean'})

degree_by_all_departments_per_student = degree_by_relevant_departments_per_student.unstack(level=-1)

degree_by_all_departments_per_student['General'] = general_degree_per_student.values
degree_by_all_departments_per_student['courses'] = courses_number_per_student.values

degree_by_all_departments_per_student.round(3)




degree                          General  \
DepartmentName                     Arts English  Science   Sport           
StudentId FirstName   LastName                                             
1.0        Alexandria Heller     86.678     NaN   79.558     NaN  81.338   
2.0        Cassie     Liska      79.543     NaN   78.867     NaN  79.137   
3.0        Hayley     Frank         NaN  83.972   76.204     NaN  78.794   
4.0        Maxwell    Kotch      69.356  70.930   74.157     NaN  71.481   
5.0        Dylan      Boehm      64.495     NaN  100.000  65.661  76.719   
...                                 ...     ...      ...     ...     ...   
275.0      Elise      Ohler      94.703  71.617   80.897     NaN  82.029   
276.0      Dylan      Bannister     NaN     NaN   76.785     NaN  76.785   
277.0      Bryan      Medina     78.849     NaN   81.299     NaN  80.482   
278.0      Sydney     Blas          NaN     NaN   72.578  76.750  75.359   
280.0      Matthew    Mcgrath    77.238     NaN   75.697     NaN  76.621   

                                courses  
DepartmentName                           
StudentId FirstName   LastName           
1.0        Alexandria Heller          4  
2.0        Cassie     Liska           5  
3.0        Hayley     Frank           3  
4.0        Maxwell    Kotch           3  
5.0        Dylan      Boehm           3  
...                                 ...  
275.0      Elise      Ohler           4  
276.0      Dylan      Bannister       3  
277.0      Bryan      Medina          6  
278.0      Sydney     Blas            3  
280.0      Matthew    Mcgrath         5  

[256 rows x 6 columns]